**输入/输出自动机**

一个I/O自动机$A$，简称为自动机$A$，由五部分组成：
1. $sig(A)$：一个签名，指定了自动机 “能做什么动作”，而不是“什么时候做”。
2. $states(A)$：一个状态集合（不一定有限）
3. $start(A)$：$states(A)$的一个非空子集，被称为起始状态或初始状态
4. $trans(A)$：状态转移关系，这里$trans(A)\subset states(A)$×$acts(sig(A))$×$states(A)$；对于任意状态$s$及任意输入动作$\pi$，都有$transitions(s,\pi,s^{'})\in trans(A)$
5. $task(A)$：一个任务分割，是$local(sig(A))$上的一个等价关系，拥有可数的等价类

DijkstraME算法：系统中始终只有一个“令牌（token）”，只有持有令牌的进程才能进入临界区，是一类基于令牌的互斥算法。

互斥：在一个由多个进程组成、无共享内存、仅靠消息通信的分布式系统中，保证：
任意时刻 最多只有一个进程进入临界区；所有请求最终都能被满足（无死锁、无饥饿）

模型假设：
1. 进程集合：进程编号：$P_0, P_1,...P_{n-1}$
2. 通信方式：点对点消息传递；消息可靠、不丢失、不重复
3. 拓扑结构：进程逻辑上排成一个环
4. 无共享变量：所有信息通过消息传递

算法规则：
1. 请求进入临界区：当进程$P_{i}$想进入临界区，
state := WANTED;
如果 has_token == true:
    state := HELD;
    进入临界区;
否则:
    等待令牌
2. 令牌的传递：当进程持有令牌并且自己不需要进入临界区（state ≠ WANTED）或者刚从临界区出来，它会：将 token 发送给 ring 中的下一个进程，令牌沿着环单向循环传递。
3. 退出临界区：当$P_{i}$退出临界区，state:=RELEASED;将token发送给下一个进程

In [1]:
from enum import Enum

class State(Enum):
    RELEASED = 0
    WANTED = 1
    HELD = 2
import threading
import queue
import time
import random
class DijkstraProcess(threading.Thread):
    def __init__(self, pid, inbox, outbox):
        super().__init__()
        self.pid = pid
        self.inbox = inbox      # 接收 token
        self.outbox = outbox    # 发送 token
        self.state = State.RELEASED
        self.has_token = False
        self.running = True

    def request_cs(self):
        print(f"P{self.pid}: requests CS")
        self.state = State.WANTED

    def enter_cs(self):
        self.state = State.HELD
        print(f"🔥 P{self.pid}: ENTER CS")
        time.sleep(random.uniform(0.5, 1.5))  # 模拟临界区
        print(f"✅ P{self.pid}: EXIT CS")
        self.state = State.RELEASED
        self.send_token()

    def send_token(self):
        self.has_token = False
        self.outbox.put("TOKEN")
        print(f"P{self.pid}: passes TOKEN")

    def handle_token(self):
        self.has_token = True
        print(f"P{self.pid}: receives TOKEN")

        if self.state == State.WANTED:
            self.enter_cs()
        else:
            # 不需要 CS，立刻传递
            time.sleep(0.1)
            self.send_token()

    def run(self):
        while self.running:
            try:
                msg = self.inbox.get(timeout=0.1)
                if msg == "TOKEN":
                    self.handle_token()
            except queue.Empty:
                pass
def run_dijkstra_me(n=5, runtime=10):
    queues = [queue.Queue() for _ in range(n)]
    processes = []

    for i in range(n):
        inbox = queues[i]
        outbox = queues[(i + 1) % n]
        p = DijkstraProcess(i, inbox, outbox)
        processes.append(p)

    # 初始 token 给 P0
    processes[0].has_token = True
    queues[0].put("TOKEN")

    # 启动进程
    for p in processes:
        p.start()

    # 随机请求临界区
    start_time = time.time()
    while time.time() - start_time < runtime:
        time.sleep(random.uniform(0.5, 1.2))
        random.choice(processes).request_cs()

    # 停止
    for p in processes:
        p.running = False
    for p in processes:
        p.join()
if __name__ == "__main__":
    run_dijkstra_me(n=4, runtime=15)


P0: receives TOKEN
P0: passes TOKEN
P1: receives TOKEN
P1: passes TOKEN
P2: receives TOKEN
P2: passes TOKEN
P3: receives TOKEN
P3: passes TOKEN
P0: receives TOKEN
P0: passes TOKEN
P1: receives TOKEN
P1: passes TOKENP2: receives TOKEN

P2: passes TOKEN
P3: receives TOKEN
P3: passes TOKEN
P0: receives TOKEN
P0: passes TOKENP1: receives TOKEN

P1: passes TOKEN
P2: receives TOKEN
P2: passes TOKEN
P3: receives TOKEN
P0: requests CS
P3: passes TOKEN
P0: receives TOKEN
🔥 P0: ENTER CS
P2: requests CS
✅ P0: EXIT CS
P0: passes TOKEN
P1: receives TOKEN
P1: passes TOKENP2: receives TOKEN
🔥 P2: ENTER CS

P2: requests CS
✅ P2: EXIT CS
P2: passes TOKEN
P3: receives TOKEN
P3: passes TOKEN
P0: receives TOKEN
P3: requests CS
P0: passes TOKEN
P1: receives TOKEN
P1: passes TOKENP2: receives TOKEN

P2: passes TOKEN
P3: receives TOKEN
🔥 P3: ENTER CS
✅ P3: EXIT CS
P3: passes TOKEN
P0: receives TOKEN
P0: passes TOKEN
P1: receives TOKEN
P1: passes TOKEN
P2: receives TOKEN
P1: requests CS
P2: passes TOKEN
P3: r

Peterson 2P算法：

算法假设：
1. 原子读写：对共享变量的读写是原子的
2. 顺序一致性：程序按源码顺序执行，不允许重排序
3. 仅限两个进程：多进程需扩展

算法分析：
1. 只有一个进程请求：$P_{i}$设置 $flag[i]=true$,$flag[j]=false$,条件不成立 → 立即进入 CS
2. 两个进程同时请求：假设$flag[0]=true,flag[1]=true$,两者都会写 turn：最后一次写决定 turn。若 turn = 0 → P0 先；若 turn = 1 → P1 先。另一个进程在 while 中等待。
3. 释放临界区：进程退出CS，$flag[i]=false$，对方立即跳出while，进入CS

In [2]:
import threading
import time
import random
flag = [False, False]
turn = 0

# 用于检测互斥是否被破坏（调试用）
in_cs = 0
lock = threading.Lock()
def peterson_process(i, rounds=5):
    global flag, turn, in_cs
    j = 1 - i

    for _ in range(rounds):
        # ---------------------------
        # Entry section
        # ---------------------------
        flag[i] = True
        turn = j
        while flag[j] and turn == j:
            pass  # busy wait

        # ---------------------------
        # Critical section
        # ---------------------------
        with lock:
            in_cs += 1
            assert in_cs == 1, "❌ Mutual exclusion violated!"
            print(f"🔥 P{i} ENTER CS")

        time.sleep(random.uniform(0.2, 0.5))

        with lock:
            print(f"✅ P{i} EXIT CS")
            in_cs -= 1

        # ---------------------------
        # Exit section
        # ---------------------------
        flag[i] = False

        # ---------------------------
        # Non-critical section
        # ---------------------------
        time.sleep(random.uniform(0.1, 0.4))
if __name__ == "__main__":
    t0 = threading.Thread(target=peterson_process, args=(0,))
    t1 = threading.Thread(target=peterson_process, args=(1,))

    t0.start()
    t1.start()

    t0.join()
    t1.join()

    print("✔ Peterson 2P simulation finished")


🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
✔ Peterson 2P simulation finished


锦标赛算法：把 N 个进程组织成一棵二叉树，每个内部节点都是一个 Peterson 2P 锁；进程从叶子一路“打赢”到根，才能进入临界区。

算法流程：对于进程$P_{i}$
1. 从叶子开始：找到自己对应的叶子,确定在每一层中：自己是左孩子(0)还是右孩子(1)
2. 逐层向上竞争：在每一层，在该层对应的 Peterson 锁中：设置 flag[side] = true;turn = other;忙等直到“赢得该层比赛”;晋级到上一层
3. 到达根节点：成为冠军，进入临界区
4. 退出临界区时：反向释放锁，从根到叶子，逐层将$flag[side]=false$，让在下层等待的进程继续比赛


In [3]:
import math
class PetersonLock:
    def __init__(self):
        self.flag = [False, False]
        self.turn = 0

    def lock(self, i):
        j = 1 - i
        self.flag[i] = True
        self.turn = j
        while self.flag[j] and self.turn == j:
            pass  # busy wait

    def unlock(self, i):
        self.flag[i] = False
class TournamentLock:
    def __init__(self, n):
        assert n & (n - 1) == 0, "n must be a power of 2"
        self.n = n
        self.levels = int(math.log2(n))
        self.locks = []

        for level in range(self.levels):
            num_locks = n // (2 ** (level + 1))
            self.locks.append([PetersonLock() for _ in range(num_locks)])
    def lock(self, pid):
        path = []
        node = pid

        for level in range(self.levels):
            side = node % 2               # 左(0) / 右(1)
            node //= 2
            lock = self.locks[level][node]
            lock.lock(side)
            path.append((level, node, side))

        return path  # 记录路径，释放时用
    def unlock(self, path):
        for level, node, side in reversed(path):
            self.locks[level][node].unlock(side)
import threading
import time
import random
in_cs = 0
check_lock = threading.Lock()
def worker(pid, tlock, rounds=3):
    global in_cs

    for _ in range(rounds):
        time.sleep(random.uniform(0.1, 0.5))

        path = tlock.lock(pid)

        # -------- Critical Section --------
        with check_lock:
            in_cs += 1
            assert in_cs == 1, "❌ Mutual exclusion violated!"
            print(f"🔥 P{pid} ENTER CS")

        time.sleep(random.uniform(0.2, 0.6))

        with check_lock:
            print(f"✅ P{pid} EXIT CS")
            in_cs -= 1
        # ----------------------------------

        tlock.unlock(path)
if __name__ == "__main__":
    N = 4   # 必须是 2 的幂
    tlock = TournamentLock(N)

    threads = [
        threading.Thread(target=worker, args=(i, tlock))
        for i in range(N)
    ]

    for t in threads:
        t.start()

    for t in threads:
        t.join()

    print("✔ Tournament lock simulation finished")


🔥 P3 ENTER CS
✅ P3 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P2 ENTER CS
✅ P2 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
🔥 P3 ENTER CS
✅ P3 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P2 ENTER CS
✅ P2 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
🔥 P3 ENTER CS
✅ P3 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P2 ENTER CS
✅ P2 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
✔ Tournament lock simulation finished


BurnsME算法：用“优先级 + 意愿标志”，让编号小的进程在竞争中天然占优。这是一种非对称（asymmetric）互斥算法。

模型假设：目标是实现互斥访问临界区
1. N个进程$P_{0},P_{1}...P_{n-1}$
2. 共享内存模型
3. 仅使用原子读写
4. 不使用硬件原子指令

In [4]:
import threading
import time
import random
N = 4                      # 进程数
flag = [False] * N         # flag[i] = True 表示 Pi 想进入 CS

# 用于检测互斥是否被破坏
in_cs = 0
check_lock = threading.Lock()
def burns_process(i, rounds=5):
    global in_cs
    n = N

    for _ in range(rounds):
        time.sleep(random.uniform(0.1, 0.4))  # 非临界区

        while True:
            # -----------------------------
            # Entry section
            # -----------------------------
            flag[i] = True

            # 1️⃣ 等待所有高优先级进程
            for j in range(i):
                while flag[j]:
                    pass

            # 2️⃣ 检查低优先级进程（回退机制）
            restart = False
            for j in range(i + 1, n):
                if flag[j]:
                    flag[i] = False
                    while flag[j]:
                        pass
                    restart = True
                    break

            if restart:
                continue

            # -----------------------------
            # Critical section
            # -----------------------------
            with check_lock:
                in_cs += 1
                assert in_cs == 1, "❌ Mutual exclusion violated!"
                print(f"🔥 P{i} ENTER CS")

            time.sleep(random.uniform(0.2, 0.6))

            with check_lock:
                print(f"✅ P{i} EXIT CS")
                in_cs -= 1

            # -----------------------------
            # Exit section
            # -----------------------------
            flag[i] = False
            break
if __name__ == "__main__":
    threads = [
        threading.Thread(target=burns_process, args=(i,))
        for i in range(N)
    ]

    for t in threads:
        t.start()

    for t in threads:
        t.join()

    print("✔ Burns-ME simulation finished")


🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P3 ENTER CS
✅ P3 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P3 ENTER CS
✅ P3 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P3 ENTER CS
✅ P3 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P2 ENTER CS
✅ P2 EXIT CS
🔥 P3 ENTER CS
✅ P3 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
🔥 P3 ENTER CS
✅ P3 EXIT CS
🔥 P0 ENTER CS
✅ P0 EXIT CS
🔥 P2 ENTER CS
✅ P2 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
🔥 P2 ENTER CS
✅ P2 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
🔥 P2 ENTER CS
✅ P2 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
🔥 P2 ENTER CS
✅ P2 EXIT CS
🔥 P1 ENTER CS
✅ P1 EXIT CS
✔ Burns-ME simulation finished


Bakery算法：

模型假设：
1. N个进程$P_{0},P_{1}...P_{n-1}$
2. 共享内存模型
3. 仅使用原子读写
4. 不使用硬件原子指令

通过模拟取号排队机制，在仅依赖原子读写的共享内存模型下，为 N 个并发进程提供了严格互斥、无死锁且无饥饿的公平临界区访问，是多进程软件互斥算法的经典代表。

In [5]:
import threading
import time
import random
N = 4                          # 进程数
choosing = [False] * N
number = [0] * N

# 用于检测互斥是否被破坏
in_cs = 0
check_lock = threading.Lock()
def bakery_process(i, rounds=5):
    global in_cs
    n = N

    for _ in range(rounds):
        time.sleep(random.uniform(0.1, 0.4))  # 非临界区

        # -----------------------------
        # Entry section (取号)
        # -----------------------------
        choosing[i] = True
        number[i] = 1 + max(number)
        choosing[i] = False

        # -----------------------------
        # 等待排在前面的进程
        # -----------------------------
        for j in range(n):
            if j == i:
                continue

            # 等对方取号完成
            while choosing[j]:
                pass

            # 等排在我前面的进程
            while number[j] != 0 and (number[j], j) < (number[i], i):
                pass

        # -----------------------------
        # Critical section
        # -----------------------------
        with check_lock:
            in_cs += 1
            assert in_cs == 1, "❌ Mutual exclusion violated!"
            print(f"🔥 P{i} ENTER CS (ticket={number[i]})")

        time.sleep(random.uniform(0.2, 0.6))

        with check_lock:
            print(f"✅ P{i} EXIT CS")
            in_cs -= 1

        # -----------------------------
        # Exit section
        # -----------------------------
        number[i] = 0
if __name__ == "__main__":
    threads = [
        threading.Thread(target=bakery_process, args=(i,))
        for i in range(N)
    ]

    for t in threads:
        t.start()

    for t in threads:
        t.join()

    print("✔ Bakery algorithm simulation finished")


🔥 P3 ENTER CS (ticket=1)
✅ P3 EXIT CS
🔥 P2 ENTER CS (ticket=2)
✅ P2 EXIT CS
🔥 P1 ENTER CS (ticket=3)
✅ P1 EXIT CS
🔥 P0 ENTER CS (ticket=4)
✅ P0 EXIT CS
🔥 P3 ENTER CS (ticket=5)
✅ P3 EXIT CS
🔥 P2 ENTER CS (ticket=6)
✅ P2 EXIT CS
🔥 P1 ENTER CS (ticket=7)
✅ P1 EXIT CS
🔥 P0 ENTER CS (ticket=8)
✅ P0 EXIT CS
🔥 P3 ENTER CS (ticket=9)
✅ P3 EXIT CS
🔥 P2 ENTER CS (ticket=10)
✅ P2 EXIT CS
🔥 P1 ENTER CS (ticket=11)
✅ P1 EXIT CS
🔥 P0 ENTER CS (ticket=12)
✅ P0 EXIT CS
🔥 P3 ENTER CS (ticket=13)
✅ P3 EXIT CS
🔥 P2 ENTER CS (ticket=14)
✅ P2 EXIT CS
🔥 P1 ENTER CS (ticket=15)
✅ P1 EXIT CS
🔥 P0 ENTER CS (ticket=16)
✅ P0 EXIT CS
🔥 P3 ENTER CS (ticket=17)
✅ P3 EXIT CS
🔥 P2 ENTER CS (ticket=18)
✅ P2 EXIT CS
🔥 P1 ENTER CS (ticket=19)
✅ P1 EXIT CS
🔥 P0 ENTER CS (ticket=20)
✅ P0 EXIT CS
✔ Bakery algorithm simulation finished


TicketME算法：类似面包店算法，但不需要 choosing 数组，是 QueueME 的正式化和抽象。

伪代码：


```plaintext
# Entry section
my_ticket = next_ticket
next_ticket += 1
while now_serving != my_ticket:
    ;  # busy wait

critical section

# Exit section
now_serving += 1




异步环境中的一致性：

在异步环境下，一致性关注的是多个进程对共享数据或状态的“看法”是否保持正确。
定义为：对系统中的共享数据/资源，任意时间点，所有进程观察到的行为或结果符合某种约定的规范。

关键机制：
1. 逻辑时钟：给每个事件一个逻辑时间戳，用于维护操作因果关系
2. 向量时钟：用数组记录每个进程的事件计数，可以判断两个事件是否因果相关或并发
3. 同步协议：Paxos、Raft 等协议实现线性一致性，用于解决异步消息延迟导致的写冲突
4. Ticket：弱一致性系统常用，用于最终一致性，写操作附带版本号，读取时选最大版本

RMWAgreement算法：系统中由$N$个进程$P_{0},P_{1},...P_{N-1}$，共享对象：单个变量$X$，每个进程都有一个提议值$v_{i}$。目标：所有进程通过对共享对象的操作达成 一致决策值

模型假设：
1. 共享内存模型
2. 提供RMW操作
3. 异步环境：进程速度不固定，执行顺序不可预测
4. 进程可能崩溃或延迟

核心思想：
1. 使用共享对象实现决策传播：
每个进程通过 原子 RMW 读写共享对象；第一个写入共享对象的值，作为全局一致决策
2. 后续进程读取共享对象：如果共享对象已有值，则决定该值；如果共享对象为空，则写入自己的提议值

伪代码：
```plaintxt
Input: v_i  # Pi 的提议值
Output: decision

1. old_value = X.swap(v_i)  # 原子读-写
2. if old_value == ⊥:       # 没有人写过
       decision = v_i
   else:
       decision = old_value
3. return decision
```

In [6]:
import threading
import random
import time

# -----------------------------
# 全局共享对象
# -----------------------------
N = 5  # 进程数
decision_value = None
rmw_lock = threading.Lock()  # 模拟原子 RMW

# 用于记录每个进程决策
decisions = [None] * N
def rmw_process(pid, proposed_value):
    global decision_value, decisions

    # 模拟非临界区延迟
    time.sleep(random.uniform(0.1, 0.5))

    # -----------------------------
    # 原子 RMW
    # -----------------------------
    with rmw_lock:
        old_value = decision_value
        if old_value is None:
            # 第一次写入
            decision_value = proposed_value
            decisions[pid] = proposed_value
            print(f"Process {pid} sets decision = {proposed_value}")
        else:
            # 已经有人写过
            decisions[pid] = old_value
            print(f"Process {pid} adopts decision = {old_value}")
if __name__ == "__main__":
    # 每个进程提议不同值
    proposed_values = [10, 20, 30, 40, 50]

    threads = [
        threading.Thread(target=rmw_process, args=(i, proposed_values[i]))
        for i in range(N)
    ]

    for t in threads:
        t.start()

    for t in threads:
        t.join()

    print("\nAll decisions:", decisions)
    print("Global decision_value:", decision_value)


Process 3 sets decision = 40
Process 1 adopts decision = 40
Process 0 adopts decision = 40
Process 4 adopts decision = 40
Process 2 adopts decision = 40

All decisions: [40, 40, 40, 40, 40]
Global decision_value: 40
